# Brisbane Burbs Best Business Battle
### Determining what type of business would be most successful in each Brisbane suburb.

This notebook attempts to determine what type of business might be best suitable for each suburb in Brisbane, Australia.

## Introduction / Business Problem

This notebook is aimed towards business entrepeneurs, or potential franchisees, who are looking to set up a new business in any of the suburbs around Brisbane, Australia. There are already a number of well established businesses in Brisbane, so this notebook attempts to analyze what type of businesses are most common in each suburb. The entrepreneur can then make a more informed desicion as to what type of business, per suburb, might be most successful.  

## Target Audience

Anyone looking to start a new business/franchise in the Brisbane area, be it a new restaurant, cafe, gym, or yoga studio, etc.

## Data

In order to solve this problem, two sets of data will be used. The first set of data contains the names of each suburb in Brisbane, the post code, the LGA, and the geospatial coordinates (longitude and latitude) of each suburb. This set of data is scraped using BeutifulSoup from the URL: https://www.geonames.org/postal-codes/AU/QLD/queensland.html. The data is also cleaned and formatted into a pandas dataframe, so it can be used with the second set of data in order to find insight. 

The second set of data is from FourSquare, which contains location data such as venue name and venue category, based on the geospatial coordinates of the suburbs. The FourSquare API is used to retreive this data, using a free FourSquare developer account, which can be created here: https://developer.foursquare.com/. 

The two datasets are then explored and analyzed together to find insight to the data. For example, the top 5 most frequent type of venue per suburb is found, as well as clusters of venue types using the kmeans algorithm.

### Retreiving the first dataset, and cleaning/formatting it into a clean dataframe format.

Import required libraries to retreive the data

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup 

Scrape the URL to retreive the raw data used for the first set of data, and load it into a dataframe.

In [11]:
req = requests.get("https://www.geonames.org/postal-codes/AU/QLD/queensland.html")

soup = BeautifulSoup(req.content,'lxml')

table = soup.find_all('table')[2]

df = pd.read_html(str(table))

brisbane_df=pd.DataFrame(df[0])
brisbane_df

,Unnamed: 0,Place,Code,Country,Admin1,Admin2,Admin3
0,1.0,Spring Hill,4000,Australia,Queensland,BRIS CITY CNTRY,NaN
1,NaN,-27.462/153.025,-27.462/153.025,-27.462/153.025,-27.462/153.025,-27.462/153.025,-27.462/153.025
2,2.0,Brisbane City,4000,Australia,Queensland,NaN,NaN
3,NaN,-27.468/153.027,-27.468/153.027,-27.468/153.027,-27.468/153.027,-27.468/153.027,-27.468/153.027
4,3.0,Hamilton,4007,Australia,Queensland,NORTHGATE CENT,NaN
...,...,...,...,...,...,...,...
396,199.0,Archerfield,4108,Australia,Queensland,UNDERWOOD CENT,NaN
397,NaN,-27.565/153.017,-27.565/153.017,-27.565/153.017,-27.565/153.017,-27.565/153.017,-27.565/153.017
398,200.0,Robertson,4109,Australia,Queensland,UNDERWOOD CENT,NaN
399,NaN,-27.565/153.056,-27.565/153.056,-27.565/153.056,-27.565/153.056,-27.565/153.056,-27.565/153.056


As can be seen above, the location data scraped show up on its own row (every other row). Let's clean this up so we have the latitude and longitude coordinates of each suburb in the same row.

The rows that contain 'NaN' values in column 'Unnamed: 0' has the coordinate values. Let's filter them out.

In [12]:
geo_data = brisbane_df['Unnamed: 0'].isna()
geo_data.head()

0    False
1     True
2    False
3     True
4    False
Name: Unnamed: 0, dtype: bool

Now we can create a dataframe that only contains the geospatial coordinates.

In [13]:
geo_df = brisbane_df[geo_data]
geo_df

,Unnamed: 0,Place,Code,Country,Admin1,Admin2,Admin3
1,NaN,-27.462/153.025,-27.462/153.025,-27.462/153.025,-27.462/153.025,-27.462/153.025,-27.462/153.025
3,NaN,-27.468/153.027,-27.468/153.027,-27.468/153.027,-27.468/153.027,-27.468/153.027,-27.468/153.027
5,NaN,-27.438/153.066,-27.438/153.066,-27.438/153.066,-27.438/153.066,-27.438/153.066,-27.438/153.066
7,NaN,-27.419/153.054,-27.419/153.054,-27.419/153.054,-27.419/153.054,-27.419/153.054,-27.419/153.054
9,NaN,-27.402/153.062,-27.402/153.062,-27.402/153.062,-27.402/153.062,-27.402/153.062,-27.402/153.062
...,...,...,...,...,...,...,...
393,NaN,-27.552/153.034,-27.552/153.034,-27.552/153.034,-27.552/153.034,-27.552/153.034,-27.552/153.034
395,NaN,-27.567/153.039,-27.567/153.039,-27.567/153.039,-27.567/153.039,-27.567/153.039,-27.567/153.039
397,NaN,-27.565/153.017,-27.565/153.017,-27.565/153.017,-27.565/153.017,-27.565/153.017,-27.565/153.017
399,NaN,-27.565/153.056,-27.565/153.056,-27.565/153.056,-27.565/153.056,-27.565/153.056,-27.565/153.056


Now let's remove (drop) the rows that contain 'NaN' values from our original dataframe. But first, let's drop the last column ('Admin3') as we don't need it.

In [14]:
brisbane_df.drop(['Admin3'], axis=1, inplace=True)
brisbane_df.dropna(how="all",inplace=True)

In [19]:
brisbane_df.dropna(how='any', inplace=True, axis=0 )
brisbane_df

,Place,Code,Country,Admin1,Admin2
0,Spring Hill,4000,Australia,Queensland,BRIS CITY CNTRY
4,Hamilton,4007,Australia,Queensland,NORTHGATE CENT
6,Clayfield,4011,Australia,Queensland,NORTHGATE CENT
8,Nundah,4012,Australia,Queensland,NORTHGATE NORTH
10,Bracken Ridge,4017,Australia,Queensland,NORTHGATE CENT
...,...,...,...,...,...
390,Rocklea,4106,Australia,Queensland,UNDERWOOD CENT
392,Salisbury,4107,Australia,Queensland,UNDERWOOD CENT
394,Coopers Plains,4108,Australia,Queensland,UNDERWOOD CENT
396,Archerfield,4108,Australia,Queensland,UNDERWOOD CENT


Let's also drop the first column ('Unnamed: 0') as we don't need that anymore either.

In [18]:
brisbane_df.drop(['Unnamed: 0'], axis=1, inplace=True)
brisbane_df.dropna(how="all",inplace=True)
brisbane_df

,Place,Code,Country,Admin1,Admin2
0,Spring Hill,4000,Australia,Queensland,BRIS CITY CNTRY
4,Hamilton,4007,Australia,Queensland,NORTHGATE CENT
6,Clayfield,4011,Australia,Queensland,NORTHGATE CENT
8,Nundah,4012,Australia,Queensland,NORTHGATE NORTH
10,Bracken Ridge,4017,Australia,Queensland,NORTHGATE CENT
...,...,...,...,...,...
390,Rocklea,4106,Australia,Queensland,UNDERWOOD CENT
392,Salisbury,4107,Australia,Queensland,UNDERWOOD CENT
394,Coopers Plains,4108,Australia,Queensland,UNDERWOOD CENT
396,Archerfield,4108,Australia,Queensland,UNDERWOOD CENT


Let's also reset the index of the dataframe.

In [20]:
brisbane_df.reset_index(drop=True, inplace=True)
brisbane_df

,Place,Code,Country,Admin1,Admin2
0,Spring Hill,4000,Australia,Queensland,BRIS CITY CNTRY
1,Hamilton,4007,Australia,Queensland,NORTHGATE CENT
2,Clayfield,4011,Australia,Queensland,NORTHGATE CENT
3,Nundah,4012,Australia,Queensland,NORTHGATE NORTH
4,Bracken Ridge,4017,Australia,Queensland,NORTHGATE CENT
...,...,...,...,...,...
191,Rocklea,4106,Australia,Queensland,UNDERWOOD CENT
192,Salisbury,4107,Australia,Queensland,UNDERWOOD CENT
193,Coopers Plains,4108,Australia,Queensland,UNDERWOOD CENT
194,Archerfield,4108,Australia,Queensland,UNDERWOOD CENT


Since the latitude and longitude values are in the same column, let's split them into separate columns and label them 'longitude' and 'latitude' respectively.

In [22]:
geo_df = geo_df.Place.str.split(pat='/', expand=True)
geo_df

,0,1
1,-27.462,153.025
3,-27.468,153.027
5,-27.438,153.066
7,-27.419,153.054
9,-27.402,153.062
...,...,...
393,-27.552,153.034
395,-27.567,153.039
397,-27.565,153.017
399,-27.565,153.056


In [23]:
geo_df.rename(columns={0:'latitude',
                      1:'longitude'}, inplace=True)
geo_df.columns

Index(['latitude', 'longitude'], dtype='object')

Let's alos reset the index.

In [24]:
geo_df.reset_index(drop=True, inplace=True)
geo_df

,latitude,longitude
0,-27.462,153.025
1,-27.468,153.027
2,-27.438,153.066
3,-27.419,153.054
4,-27.402,153.062
...,...,...
196,-27.552,153.034
197,-27.567,153.039
198,-27.565,153.017
199,-27.565,153.056


Finally, let's combine the two dataframes into a final, merged dataframe that contains both the suburb names, codes, etc. as well as the geospatial coordinates.

In [25]:
brisbane_geo_df = pd.concat([brisbane_df, geo_df], axis=1, sort=False)
brisbane_geo_df

,Place,Code,Country,Admin1,Admin2,latitude,longitude
0,Spring Hill,4000,Australia,Queensland,BRIS CITY CNTRY,-27.462,153.025
1,Hamilton,4007,Australia,Queensland,NORTHGATE CENT,-27.468,153.027
2,Clayfield,4011,Australia,Queensland,NORTHGATE CENT,-27.438,153.066
3,Nundah,4012,Australia,Queensland,NORTHGATE NORTH,-27.419,153.054
4,Bracken Ridge,4017,Australia,Queensland,NORTHGATE CENT,-27.402,153.062
...,...,...,...,...,...,...,...
196,NaN,NaN,NaN,NaN,NaN,-27.552,153.034
197,NaN,NaN,NaN,NaN,NaN,-27.567,153.039
198,NaN,NaN,NaN,NaN,NaN,-27.565,153.017
199,NaN,NaN,NaN,NaN,NaN,-27.565,153.056


In [29]:
brisbane_geo_df.shape

(196, 7)

Let's also label the columns more appropriately.

In [30]:
brisbane_geo_df.rename(columns={'Place':'Suburb',
                                'Admin1':'State',
                                'Admin2':'LGA'
                               }, inplace=True)
brisbane_geo_df.columns

Index(['Suburb', 'Code', 'Country', 'State', 'LGA', 'latitude', 'longitude'], dtype='object')

And also drop any remaining rows with 'NaN' values. Finally we have our first dataset in a dataframe format.

In [28]:
brisbane_geo_df.dropna(inplace=True)
brisbane_geo_df

,Suburb,Code,Country,State,LGA,latitude,longitude
0,Spring Hill,4000,Australia,Queensland,BRIS CITY CNTRY,-27.462,153.025
1,Hamilton,4007,Australia,Queensland,NORTHGATE CENT,-27.468,153.027
2,Clayfield,4011,Australia,Queensland,NORTHGATE CENT,-27.438,153.066
3,Nundah,4012,Australia,Queensland,NORTHGATE NORTH,-27.419,153.054
4,Bracken Ridge,4017,Australia,Queensland,NORTHGATE CENT,-27.402,153.062
...,...,...,...,...,...,...,...
191,Rocklea,4106,Australia,Queensland,UNDERWOOD CENT,-27.507,153.024
192,Salisbury,4107,Australia,Queensland,UNDERWOOD CENT,-27.515,153.013
193,Coopers Plains,4108,Australia,Queensland,UNDERWOOD CENT,-27.525,153.016
194,Archerfield,4108,Australia,Queensland,UNDERWOOD CENT,-27.525,153.004


In [31]:
brisbane_geo_df.shape

(196, 7)

### Retreiving the second dataset from FourSquare, via their API.

Importing the required libraries.

In [78]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import json # library to handle JSON files

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab

import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


Solving environment: done

# All requested packages already installed.

Libraries imported.


Exploring the Brisbane, Australia location.

In [33]:
address = 'Brisbane, Australia'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Brisbane, Australia are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Brisbane, Australia are -27.4689682, 153.0234991.


Defining the FourSquare API credentials.

In [35]:
CLIENT_ID = 'QKAHEWSRIS0OJH5TAJUAZPYTUR3AAZAM1UFYOY2FBHKH0G1P' # your Foursquare ID
CLIENT_SECRET = 'FUN0ESQLCT52Z5PKNZV3G1MCZ3KKCNJ20KWWWTFJ2V52GF3C' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QKAHEWSRIS0OJH5TAJUAZPYTUR3AAZAM1UFYOY2FBHKH0G1P
CLIENT_SECRET:FUN0ESQLCT52Z5PKNZV3G1MCZ3KKCNJ20KWWWTFJ2V52GF3C


Let's explore the first suburb in the dataframe.

In [36]:
brisbane_geo_df.loc[0, 'Suburb']

'Spring Hill'

In [37]:
suburb_latitude = brisbane_geo_df.loc[0, 'latitude'] # neighborhood latitude value
suburb_longitude = brisbane_geo_df.loc[0, 'longitude'] # neighborhood longitude value

suburb_name = brisbane_geo_df.loc[0, 'Suburb'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(suburb_name, 
                                                               suburb_latitude, 
                                                               suburb_longitude))

Latitude and longitude values of Spring Hill are -27.462, 153.025.


Now let's define the variables used to retreive data from the FourSquare API.

In [38]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    suburb_latitude, 
    suburb_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=QKAHEWSRIS0OJH5TAJUAZPYTUR3AAZAM1UFYOY2FBHKH0G1P&client_secret=FUN0ESQLCT52Z5PKNZV3G1MCZ3KKCNJ20KWWWTFJ2V52GF3C&v=20180605&ll=-27.462,153.025&radius=500&limit=100'

And retreive the results.

In [39]:
results = requests.get(url).json()

In [40]:
results

{'meta': {'code': 200, 'requestId': '5ecf1798949393001c555725'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Spring Hill',
  'headerFullLocation': 'Spring Hill, Brisbane',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 36,
  'suggestedBounds': {'ne': {'lat': -27.457499995499994,
    'lng': 153.030062004985},
   'sw': {'lat': -27.466500004500006, 'lng': 153.019937995015}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '527e183911d2a639621d2310',
       'name': 'jak + hill',
       'location': {'address': '40 Astor Tce',
        'lat': -27.4636262353871,
        'lng': 153.0256167938703,
        'labeledLatLngs': [{'label': 'display',
          'lat': -27.46362

Next, let's extract the venue category for each venue.

In [41]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [42]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

/home/joakim/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,jak + hill,Café,-27.463626,153.025617
1,Punthill Apartment Hotels,Hotel,-27.463598,153.025617
2,The Alliance Hotel,Gastropub,-27.461272,153.026370
3,Bishamon Japanese Restaurant,Japanese Restaurant,-27.460428,153.021727
4,Sisco Bcl,Café,-27.460456,153.021788
5,Domino's Pizza,Pizza Place,-27.461136,153.024413
6,WickEd Corner Cafe,Café,-27.465096,153.024788
7,Spring Hill Deli,Deli / Bodega,-27.460371,153.021384
8,St Paul's Tavern,Pub,-27.461825,153.026383
9,Toobirds Bistro & Bar | Expresso & Eats,Bistro,-27.465776,153.027283


In [43]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

36 venues were returned by Foursquare.


## Exploratory Data Analysis

Now that we have the required data, let's do some Exploratory Data Analysis (EDA).

First, let's get the nearby venues in a radius of 500 meters.

In [44]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Suburb', 
                  'Suburb Latitude', 
                  'Suburb Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [45]:
brisbane_venues = getNearbyVenues(names=brisbane_geo_df['Suburb'],
                                   latitudes=brisbane_geo_df['latitude'],
                                   longitudes=brisbane_geo_df['longitude']
                                  )

Spring Hill
Hamilton
Clayfield
Nundah
Bracken Ridge
Redcliffe
Kedron
Chermside
Aspley
Albany Creek
Kelvin Grove
Ashgrove
The Gap
Paddington
Toowong
St Lucia
Taringa
Indooroopilly
Inala
Forest Lake
West End
Woolloongabba
Annerley
Moorooka
Sunnybank Hills
Runcorn
Eight Mile Plains
Woodridge
Calamvale
Greenslopes
Upper Mount Gravatt
Mount Gravatt East
Slacks Creek
Coorparoo
Camp Hill
Carina
Carindale
Capalaba
Birkdale
Alexandra Hills
Thornlands
Redland Bay
Victoria Point
Wynnum
Wynnum West
Russell Island
Eagleby
Ormeau
Coomera
Pimpama
Upper Coomera
Oxenford
Nerang
Pacific Pines
Carrara
Hope Island
Helensvale
Mudgeeraba
Ashmore
Arundel
Labrador
Southport
Runaway Bay
Coombabah
Main Beach
Surfers Paradise
Mermaid Waters
Mermaid Beach
Broadbeach
Burleigh Waters
Miami
Burleigh Heads
Palm Beach
Coolangatta
Robina
Varsity Lakes
Jimboomba
Springfield Lakes
Redbank Plains
Warwick
Dalby
Kallangur
Narangba
Burpengary
Morayfield
Bongaree
Deception Bay
Caboolture
Buderim
Mooloolaba
Maroochydore
Nambou

In [46]:
print(brisbane_venues.shape)
brisbane_venues.head(5)

(1589, 7)


,Suburb,Suburb Latitude,Suburb Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Spring Hill,-27.462,153.025,jak + hill,-27.463626,153.025617,Café
1,Spring Hill,-27.462,153.025,Punthill Apartment Hotels,-27.463598,153.025617,Hotel
2,Spring Hill,-27.462,153.025,The Alliance Hotel,-27.461272,153.026370,Gastropub
3,Spring Hill,-27.462,153.025,Bishamon Japanese Restaurant,-27.460428,153.021727,Japanese Restaurant
4,Spring Hill,-27.462,153.025,Sisco Bcl,-27.460456,153.021788,Café


Next we'll count the number of venues for each suburb.

In [51]:
brisbane_venues_group_count = brisbane_venues.groupby('Suburb').count()
brisbane_venues_group_count

,Suburb Latitude,Suburb Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Suburb,,,,,,
Albany Creek,13,13,13,13,13,13
Albion,12,12,12,12,12,12
Alexandra Hills,3,3,3,3,3,3
Annerley,19,19,19,19,19,19
Arana Hills,7,7,7,7,7,7
...,...,...,...,...,...,...
Wynnum,2,2,2,2,2,2
Wynnum West,6,6,6,6,6,6
Yeerongpilly,4,4,4,4,4,4


In [52]:
print('There are {} uniques categories.'.format(len(brisbane_venues['Venue Category'].unique())))

There are 228 uniques categories.


In order to be able to use the categorical values (Venue Category) in a machine learning algorithm such as KMeans, let's convert the categorical values to binary values using one-hot encoding.

In [54]:
# one hot encoding
brisbane_onehot = pd.get_dummies(brisbane_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
brisbane_onehot['Suburb'] = brisbane_venues['Suburb'] 

# move neighborhood column to the first column
fixed_columns = [brisbane_onehot.columns[-1]] + list(brisbane_onehot.columns[:-1])
brisbane_onehot = brisbane_onehot[fixed_columns]

brisbane_onehot.head()

,Suburb,American Restaurant,Antique Shop,Arcade,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Spring Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Spring Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Spring Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Spring Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Spring Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
brisbane_onehot.shape

(1589, 229)

Now let's group the one-hot encoded categories by suburb, and take the mean, to find the frequency.

In [56]:
brisbane_grouped = brisbane_onehot.groupby('Suburb').mean().reset_index()
brisbane_grouped

,Suburb,American Restaurant,Antique Shop,Arcade,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Albany Creek,0.0,0.0,0.0,0.0,0.0,0.076923,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,Albion,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,Alexandra Hills,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,Annerley,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.052632,0.0,0.0,0.052632,0.0,0.0,0.0,0.0,0.0,0.0
4,Arana Hills,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,Wynnum,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
179,Wynnum West,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
180,Yeerongpilly,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
181,Yeronga,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
brisbane_grouped.shape

(183, 229)

Next we'll find the top 5 most frequent venues per suburb.

In [58]:
num_top_venues = 5

for burb in brisbane_grouped['Suburb']:
    print("----"+burb+"----")
    temp = brisbane_grouped[brisbane_grouped['Suburb'] == burb].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Albany Creek----
               venue  freq
0        Pizza Place  0.15
1   Sushi Restaurant  0.08
2  Electronics Store  0.08
3                Gym  0.08
4      Grocery Store  0.08


----Albion----
                    venue  freq
0  Furniture / Home Store  0.17
1           Grocery Store  0.08
2      Mexican Restaurant  0.08
3       German Restaurant  0.08
4             Music Store  0.08


----Alexandra Hills----
                 venue  freq
0   Chinese Restaurant  0.33
1          Pizza Place  0.33
2                  Gym  0.33
3  American Restaurant  0.00
4        Movie Theater  0.00


----Annerley----
                           venue  freq
0                    Coffee Shop  0.11
1                           Café  0.11
2  Vegetarian / Vegan Restaurant  0.05
3                 Sandwich Place  0.05
4  Paper / Office Supplies Store  0.05


----Arana Hills----
              venue  freq
0  Swiss Restaurant  0.14
1      Burger Joint  0.14
2    Farmers Market  0.14
3   Thai Restaurant  0.14
4  

Next we'll group the top 10 most frequent venues for each suburb into a single dataframe.

In [70]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [71]:
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Suburb']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
suburb_venues_sorted = pd.DataFrame(columns=columns)
suburb_venues_sorted['Suburb'] = brisbane_grouped['Suburb']

for ind in np.arange(brisbane_grouped.shape[0]):
    suburb_venues_sorted.iloc[ind, 1:] = return_most_common_venues(brisbane_grouped.iloc[ind, :], num_top_venues)

suburb_venues_sorted.shape

(183, 11)

In [72]:
suburb_venues_sorted.dropna(inplace=True)
suburb_venues_sorted

,Suburb,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Albany Creek,Pizza Place,Ice Cream Shop,Convenience Store,Sushi Restaurant,Martial Arts Dojo,Bus Station,Grocery Store,Gym,Electronics Store,Fast Food Restaurant
1,Albion,Furniture / Home Store,Mexican Restaurant,Gym,Café,Event Space,Train Station,Baby Store,Music Store,Grocery Store,German Restaurant
2,Alexandra Hills,Pizza Place,Chinese Restaurant,Gym,Yoga Studio,Field,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium,Food Truck
3,Annerley,Café,Coffee Shop,Indian Restaurant,Train Station,Pharmacy,Sandwich Place,Electronics Store,Laser Tag,Grocery Store,Bakery
4,Arana Hills,Grocery Store,Swiss Restaurant,Burger Joint,Coffee Shop,Thai Restaurant,Seafood Restaurant,Farmers Market,Yoga Studio,French Restaurant,Football Stadium
...,...,...,...,...,...,...,...,...,...,...,...
178,Wynnum,Gym / Fitness Center,Café,Yoga Studio,Fish & Chips Shop,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium,Food Truck
179,Wynnum West,Train Station,Bar,Fried Chicken Joint,Coffee Shop,Grocery Store,Fast Food Restaurant,Frozen Yogurt Shop,French Restaurant,Football Stadium,Food Truck
180,Yeerongpilly,Indian Restaurant,Café,Park,Bakery,Field,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium,Food Truck
181,Yeronga,Garden Center,Bus Station,Garden,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium,Food Truck,Food Court


## Clustering Neighborhoods¶

Let's find clusters of venue types, using the k-means algorithm.

In [73]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

brisbane_grouped_clustering = brisbane_grouped.drop('Suburb', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(brisbane_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 4, 4, 4, 4, 4, 4, 4, 4, 2], dtype=int32)

In [74]:
# add clustering labels
suburb_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

brisbane_merged = brisbane_geo_df

# merge brisbane_grouped with toronto_data to add latitude/longitude for each Suburb
brisbane_merged = brisbane_merged.join(suburb_venues_sorted.set_index('Suburb'), on='Suburb')

brisbane_merged.shape
brisbane_merged.head(10) # check the last columns!

,Suburb,Code,Country,State,LGA,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Spring Hill,4000,Australia,Queensland,BRIS CITY CNTRY,-27.462,153.025,0.0,Hotel,Café,Bar,Hotel Bar,Coffee Shop,Pub,Bus Station,Seafood Restaurant,Sandwich Place,Gastropub
1,Hamilton,4007,Australia,Queensland,NORTHGATE CENT,-27.468,153.027,4.0,Café,Coffee Shop,Hotel,Bar,Japanese Restaurant,Burger Joint,Sushi Restaurant,Restaurant,Clothing Store,Italian Restaurant
2,Clayfield,4011,Australia,Queensland,NORTHGATE CENT,-27.438,153.066,4.0,Café,Pizza Place,Seafood Restaurant,Japanese Restaurant,Italian Restaurant,Grocery Store,Convenience Store,Supermarket,Pub,Sushi Restaurant
3,Nundah,4012,Australia,Queensland,NORTHGATE NORTH,-27.419,153.054,0.0,Café,Sushi Restaurant,Portuguese Restaurant,Pharmacy,Gym,Train Station,Yoga Studio,Fish Market,Field,Fish & Chips Shop
4,Bracken Ridge,4017,Australia,Queensland,NORTHGATE CENT,-27.402,153.062,4.0,Café,Pizza Place,Yoga Studio,Japanese Restaurant,Flea Market,Pharmacy,Supermarket,Sandwich Place,Fast Food Restaurant,Garden Center
5,Redcliffe,4020,Australia,Queensland,NORTHGATE CENT,-27.322,153.032,2.0,Park,Yoga Studio,Fast Food Restaurant,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium,Food Truck,Food Court,Flower Shop
6,Kedron,4031,Australia,Queensland,NORTHGATE CENT,-27.228,153.11,4.0,Steakhouse,Italian Restaurant,Sandwich Place,Liquor Store,Fast Food Restaurant,Grocery Store,Coffee Shop,Public Art,Pizza Place,Convenience Store
7,Chermside,4032,Australia,Queensland,NORTHGATE NORTH,-27.404,153.031,4.0,Pizza Place,Convenience Store,Supermarket,Bowling Alley,Bus Station,Bus Stop,Sandwich Place,Café,Coffee Shop,Fast Food Restaurant
8,Aspley,4034,Australia,Queensland,NORTHGATE NORTH,-27.386,153.031,4.0,Fast Food Restaurant,Department Store,Café,Multiplex,Fried Chicken Joint,Coffee Shop,Big Box Store,German Restaurant,Movie Theater,Chinese Restaurant
9,Albany Creek,4035,Australia,Queensland,NORTHGATE NORTH,-27.366,153.017,4.0,Pizza Place,Ice Cream Shop,Convenience Store,Sushi Restaurant,Martial Arts Dojo,Bus Station,Grocery Store,Gym,Electronics Store,Fast Food Restaurant


In [75]:
brisbane_merged.dropna(inplace=True)

Finally, let's map the different clusters using different colors on a map of Brisbane, using Folium.

In [79]:
import folium

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(brisbane_merged['latitude'], brisbane_merged['longitude'], brisbane_merged['Suburb'], brisbane_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


As can be seen on the map, the first and last cluster is more common in the center of Brisbane, whereas the other clusters are more common in the surrounding suburbs.

### Examining the venues in each cluster

Let's see what type of venues are in each of the clusters found by the KMeans algorithm.

Cluster 1

In [80]:
brisbane_merged.loc[brisbane_merged['Cluster Labels'] == 0, brisbane_merged.columns[[1] + list(range(5, brisbane_merged.shape[1]))]]

,Code,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,4000,-27.462,153.025,0.0,Hotel,Café,Bar,Hotel Bar,Coffee Shop,Pub,Bus Station,Seafood Restaurant,Sandwich Place,Gastropub
3,4012,-27.419,153.054,0.0,Café,Sushi Restaurant,Portuguese Restaurant,Pharmacy,Gym,Train Station,Yoga Studio,Fish Market,Field,Fish & Chips Shop
12,4061,-27.443,152.988,0.0,Café,Indian Restaurant,Convenience Store,Gym,Fruit & Vegetable Store,Park,Chinese Restaurant,Ice Cream Shop,Donut Shop,Fried Chicken Joint
14,4066,-27.461,152.999,0.0,Italian Restaurant,Coffee Shop,Café,Indian Restaurant,Himalayan Restaurant,Ramen Restaurant,Shopping Mall,Toy / Game Store,Supermarket,Historic Site
23,4105,-27.512,153.032,0.0,Café,IT Services,Thai Restaurant,Coffee Shop,Restaurant,Sandwich Place,Pizza Place,Supermarket,History Museum,Football Stadium
27,4114,-27.58,153.092,0.0,Hotel,Coffee Shop,Café,Gym,Yoga Studio,Field,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium
32,4127,-27.532,153.085,0.0,Dance Studio,Video Game Store,Bus Station,Café,Field,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium,Food Truck
34,4152,-27.498,153.06,0.0,Café,Fast Food Restaurant,Coffee Shop,Burrito Place,Supermarket,Indian Restaurant,Indie Movie Theater,French Restaurant,Football Stadium,Fish Market
35,4152,-27.493,153.076,0.0,Antique Shop,Sporting Goods Shop,Bus Station,Flower Shop,Café,Light Rail Station,Yoga Studio,Fish Market,Frozen Yogurt Shop,Fried Chicken Joint
40,4164,-27.528,153.224,0.0,IT Services,Café,Thai Restaurant,Electronics Store,Liquor Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium


Cluster 2


In [81]:
brisbane_merged.loc[brisbane_merged['Cluster Labels'] == 1, brisbane_merged.columns[[1] + list(range(5, brisbane_merged.shape[1]))]]

,Code,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
144,4051,-27.339,152.947,1.0,Bus Station,Salon / Barbershop,Dessert Shop,Café,Yoga Studio,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium,Food Truck
158,4066,-27.416,152.92,1.0,Bus Station,Garden Center,Garden,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium,Food Truck,Food Court
170,4074,-27.562,152.879,1.0,Bus Station,Office,Yoga Studio,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium,Food Truck,Food Court
172,4074,-27.544,152.949,1.0,Bus Station,Yoga Studio,Garden,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium,Food Truck,Food Court
185,4101,-27.588,152.985,1.0,Bus Station,Park,Yoga Studio,Field,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium,Food Truck,Food Court
188,4104,-27.617,152.945,1.0,Garden Center,Bus Station,Garden,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium,Food Truck,Food Court


Cluster 3

In [82]:
brisbane_merged.loc[brisbane_merged['Cluster Labels'] == 2, brisbane_merged.columns[[1] + list(range(5, brisbane_merged.shape[1]))]]

,Code,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,4020,-27.322,153.032,2.0,Park,Yoga Studio,Fast Food Restaurant,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium,Food Truck,Food Court,Flower Shop
28,4116,-27.629,153.106,2.0,Supermarket,Park,Fish & Chips Shop,Fast Food Restaurant,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium,Food Truck
33,4151,-27.638,153.128,2.0,IT Services,Construction & Landscaping,Park,Fast Food Restaurant,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium,Food Truck,Food Court
41,4165,-27.562,153.266,2.0,Grocery Store,Fish & Chips Shop,Park,Yoga Studio,Fast Food Restaurant,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium,Food Truck
46,4207,-27.675,153.382,2.0,Construction & Landscaping,Yoga Studio,Field,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium,Food Truck,Food Court
56,4212,-27.869,153.36,2.0,Construction & Landscaping,Park,Yoga Studio,Fast Food Restaurant,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium,Food Truck,Food Court
58,4214,-28.088,153.363,2.0,Park,Bowling Green,Yoga Studio,Field,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium,Food Truck
78,4301,-27.68,152.919,2.0,Construction & Landscaping,Electronics Store,Yoga Studio,Field,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium,Food Truck
83,4505,-27.186,152.947,2.0,Playground,Arts & Crafts Store,Park,Trail,Fast Food Restaurant,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium,Food Truck
118,4017,-27.368,153.088,2.0,Park,Convenience Store,Train Station,Business Service,Yoga Studio,Field,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium


Cluster 4

In [83]:
brisbane_merged.loc[brisbane_merged['Cluster Labels'] == 3, brisbane_merged.columns[[1] + list(range(5, brisbane_merged.shape[1]))]]

,Code,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
85,4507,-27.117,152.946,3.0,Basketball Court,Yoga Studio,Fish & Chips Shop,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Football Stadium,Food Truck


Cluster 5

In [84]:
brisbane_merged.loc[brisbane_merged['Cluster Labels'] == 4, brisbane_merged.columns[[1] + list(range(5, brisbane_merged.shape[1]))]]

,Code,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,4007,-27.468,153.027,4.0,Café,Coffee Shop,Hotel,Bar,Japanese Restaurant,Burger Joint,Sushi Restaurant,Restaurant,Clothing Store,Italian Restaurant
2,4011,-27.438,153.066,4.0,Café,Pizza Place,Seafood Restaurant,Japanese Restaurant,Italian Restaurant,Grocery Store,Convenience Store,Supermarket,Pub,Sushi Restaurant
4,4017,-27.402,153.062,4.0,Café,Pizza Place,Yoga Studio,Japanese Restaurant,Flea Market,Pharmacy,Supermarket,Sandwich Place,Fast Food Restaurant,Garden Center
6,4031,-27.228,153.11,4.0,Steakhouse,Italian Restaurant,Sandwich Place,Liquor Store,Fast Food Restaurant,Grocery Store,Coffee Shop,Public Art,Pizza Place,Convenience Store
7,4032,-27.404,153.031,4.0,Pizza Place,Convenience Store,Supermarket,Bowling Alley,Bus Station,Bus Stop,Sandwich Place,Café,Coffee Shop,Fast Food Restaurant
8,4034,-27.386,153.031,4.0,Fast Food Restaurant,Department Store,Café,Multiplex,Fried Chicken Joint,Coffee Shop,Big Box Store,German Restaurant,Movie Theater,Chinese Restaurant
9,4035,-27.366,153.017,4.0,Pizza Place,Ice Cream Shop,Convenience Store,Sushi Restaurant,Martial Arts Dojo,Bus Station,Grocery Store,Gym,Electronics Store,Fast Food Restaurant
10,4059,-27.354,152.967,4.0,Pool,Home Service,Shopping Mall,Bus Station,Fast Food Restaurant,Fried Chicken Joint,French Restaurant,Football Stadium,Food Truck,Food Court
11,4060,-27.451,153.013,4.0,Coffee Shop,Café,Burger Joint,Park,Fast Food Restaurant,Bus Station,Sandwich Place,Thai Restaurant,Supermarket,Vietnamese Restaurant
13,4064,-27.445,152.944,4.0,Women's Store,Convenience Store,Thai Restaurant,Bus Station,Trail,Grocery Store,Fish & Chips Shop,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
